In [8]:
from nltk.tokenize.casual import casual_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import pandas as pd
import string
import datetime
import os
base = "../../data/clean/"
pd.set_option('display.max_rows', 150)

In [4]:
tweetdf = pd.read_csv(base+'loctrisma2016.csv')
tweetdf = tweetdf.sort_values(by='tweet_id')
tweetdf.head()

,Unnamed: 0,Unnamed: 0.1,tweet_id,created_at,from_user_id,source,text,to_user_id,to_user_name,in_reply_to_status_id,...,location_name,country,location_full_name,location_type,country_code,collected_at,lga,state,lat,lng
375922,412183,246385,682712842600972291,2016-01-01 00:00:00.000000,106082182,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",@legallyfm @XFilesNews @thexfiles @mynewsaga &...,39023478.0,legallyfm,6.827107e+17,...,Glen Innes,Australia,"Glen Innes, New South Wales",city,AU,2013-01-01 00:00:01,Glen Innes Severn,NSW,-29.751838,151.736189
33932,34218,503413,682712878915260418,2016-01-01 00:00:09.000000,269023896,"<a href=""http://sandaysoft.com/"" rel=""nofollow...",Temp: 33.1°C Wind: 5kph Pressure: 995.9hpa Fal...,NaN,NaN,NaN,...,Baradine,Australia,"Baradine, New South Wales",city,AU,2013-01-01 00:00:01,Warrumbungle Shire,NSW,-30.950000,149.066667
266436,291148,502347,682712884565049344,2016-01-01 00:00:10.000000,379712969,"<a href=""http://sandaysoft.com/"" rel=""nofollow...","Wind 2.7 kts NE. Barometer 1017.9 mb, Falling ...",NaN,NaN,NaN,...,Raymond Terrace,Australia,"Raymond Terrace, New South Wales",city,AU,2013-01-01 00:00:01,Port Stephens,NSW,-32.754408,151.767512
132916,145044,79326,682712982909005824,2016-01-01 00:00:33.000000,109140867,"<a href=""http://instagram.com"" rel=""nofollow"">...","Best coffee, food and swevice on the Hume and ...",NaN,NaN,NaN,...,Berremangra,Australia,"Berremangra, New South Wales",neighborhood,AU,2013-01-01 00:00:01,Hilltops,NSW,-34.830222,148.516892
565350,623155,1323531,682713033836085248,2016-01-01 00:00:46.000000,178662965,"<a href=""http://twitter.com/download/android"" ...",Best start to 2016 - a morning at my favourite...,NaN,NaN,NaN,...,Dudley,Australia,"Dudley, Newcastle",neighborhood,AU,2013-01-01 00:00:01,Lake Macquarie,NSW,-32.990000,151.718000


In [25]:
tweetdf['hour_of_day'] = tweetdf.created_at.apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S.%f").hour)

In [27]:
def time_of_day(h):
    if h in range(0,6):
        return 'night'
    elif h in range(6,12):
        return 'morning'
    elif h in range(12,18):
        return 'afternoon'
    elif h in range(18,24):
        return 'evening'

In [28]:
tweetdf['night'] = tweetdf['hour_of_day'].isin(range(0,6))
tweetdf['night'] = tweetdf['night'].astype(int)
tweetdf['morning'] = tweetdf['hour_of_day'].isin(range(6,12))
tweetdf['morning'] = tweetdf['morning'].astype(int)
tweetdf['afternoon'] = tweetdf['hour_of_day'].isin(range(12,18))
tweetdf['afternoon'] = tweetdf['afternoon'].astype(int)
tweetdf['evening'] = tweetdf['hour_of_day'].isin(range(18,24))
tweetdf['evening'] = tweetdf['evening'].astype(int)
tweetdf['time_of_day'] = tweetdf.hour_of_day.apply(time_of_day)
tweetdf.head()

,Unnamed: 0,Unnamed: 0.1,tweet_id,created_at,from_user_id,source,text,to_user_id,to_user_name,in_reply_to_status_id,...,lga,state,lat,lng,hour_of_day,night,morning,afternoon,evening,time_of_day
375922,412183,246385,682712842600972291,2016-01-01 00:00:00.000000,106082182,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",@legallyfm @XFilesNews @thexfiles @mynewsaga &...,39023478.0,legallyfm,6.827107e+17,...,Glen Innes Severn,NSW,-29.751838,151.736189,0,1,0,0,0,night
33932,34218,503413,682712878915260418,2016-01-01 00:00:09.000000,269023896,"<a href=""http://sandaysoft.com/"" rel=""nofollow...",Temp: 33.1°C Wind: 5kph Pressure: 995.9hpa Fal...,NaN,NaN,NaN,...,Warrumbungle Shire,NSW,-30.950000,149.066667,0,1,0,0,0,night
266436,291148,502347,682712884565049344,2016-01-01 00:00:10.000000,379712969,"<a href=""http://sandaysoft.com/"" rel=""nofollow...","Wind 2.7 kts NE. Barometer 1017.9 mb, Falling ...",NaN,NaN,NaN,...,Port Stephens,NSW,-32.754408,151.767512,0,1,0,0,0,night
132916,145044,79326,682712982909005824,2016-01-01 00:00:33.000000,109140867,"<a href=""http://instagram.com"" rel=""nofollow"">...","Best coffee, food and swevice on the Hume and ...",NaN,NaN,NaN,...,Hilltops,NSW,-34.830222,148.516892,0,1,0,0,0,night
565350,623155,1323531,682713033836085248,2016-01-01 00:00:46.000000,178662965,"<a href=""http://twitter.com/download/android"" ...",Best start to 2016 - a morning at my favourite...,NaN,NaN,NaN,...,Lake Macquarie,NSW,-32.990000,151.718000,0,1,0,0,0,night


In [29]:
by_time_of_day = tweetdf.groupby(['lga','time_of_day'], as_index=False).agg({'text':lambda x:' '.join(x)})
by_time_of_day

,lga,time_of_day,text
0,Albury,afternoon,"I can't say if its ""personally directed"" @ me ..."
1,Albury,evening,New year resolution \n\nFind another job so yo...
2,Albury,morning,@BluesTumut @Group9RL going to be a smashing s...
3,Albury,night,#2015bestnine🌟 #BestOfNine2015 #byebye #2015 #...
4,Armidale Regional,afternoon,#تنفيذ_القصاص_على_47_ارهابي\n\nليسوا على مذهب ...
5,Armidale Regional,evening,@Photos_KP bit of a rare site over there.. @Ph...
6,Armidale Regional,morning,@RebeccaRacer woohoo! I think there's also a n...
7,Armidale Regional,night,Tranquility in the Park. @ Moore Park Inn Bout...
8,Ballina,afternoon,When my lil sister calls her close friends cou...
9,Ballina,evening,If you are feeling too positive about yourself...


In [30]:
import string
import emoji
from unidecode import unidecode
from tokenizer import tokenizer
# Remove handles and urls but not hash tags.
# Handles might be useful for an is-reply feature but not for analysis of the text
smc_tokenizer = tokenizer.TweetTokenizer(preserve_handles=False, preserve_hashes=True, preserve_len=False,
                                         preserve_case=False, preserve_url=False)


In [32]:
# First, remove all emojis, then decode all unicode, then put emojis at the end
docs = []
for doc in by_time_of_day['text']:
    emojis = ' '.join([c for c in doc if c in emoji.UNICODE_EMOJI])
    stripped = ''.join([c for c in doc if c not in emoji.UNICODE_EMOJI])
    decoded = unidecode(stripped)
    docs.append(' '.join([decoded, emojis]))
len(docs)

524

In [33]:
# TweetTokenizer weirdly keeps punctuation separated by space as one token:
puncts = set(string.punctuation + ' ')
docs = [smc_tokenizer.tokenize(doc) for doc in docs]
cleaned = []
for doc in docs:
    clean = [token for token in doc if not(len(token) == 1 and token not in emoji.UNICODE_EMOJI)
                and not(all(c in puncts for c in token))]
    cleaned.append(' '.join(clean))

In [11]:
cleaned[0]

['who actually thinks it is ok to bring mccafe drink into normal cafe who actually thinks it is ok to bring mccafe drink into normal cafe',
 'road trip playlist happy friday my funky bunch #nowplaying #playitloud',
 'amazing day working with the film crew to create online video resources']

In [34]:
# Add cleaned tokens as an attribute to by_tweeter:
cs = pd.Series(cleaned)
by_time_of_day['cleaned'] = cs
by_time_of_day

,lga,time_of_day,text,cleaned
0,Albury,afternoon,"I can't say if its ""personally directed"" @ me ...",can't say if its personally directed me but wh...
1,Albury,evening,New year resolution \n\nFind another job so yo...,new year resolution find another job so you do...
2,Albury,morning,@BluesTumut @Group9RL going to be a smashing s...,going to be smashing season this year good luc...
3,Albury,night,#2015bestnine🌟 #BestOfNine2015 #byebye #2015 #...,#2015bestnine #bestofnine2015 #byebye #2015 #h...
4,Armidale Regional,afternoon,#تنفيذ_القصاص_على_47_ارهابي\n\nليسوا على مذهب ...,#tnfydh_lqss_ _47_rhby lysw mdhhb whd wl jnsy ...
5,Armidale Regional,evening,@Photos_KP bit of a rare site over there.. @Ph...,bit of rare site over there can't it's overcas...
6,Armidale Regional,morning,@RebeccaRacer woohoo! I think there's also a n...,woohoo think there's also number of people exc...
7,Armidale Regional,night,Tranquility in the Park. @ Moore Park Inn Bout...,tranquility in the park moore park inn boutiqu...
8,Ballina,afternoon,When my lil sister calls her close friends cou...,when my lil sister calls her close friends cou...
9,Ballina,evening,If you are feeling too positive about yourself...,if you are feeling too positive about yourself...


In [36]:
by_time_of_day.to_csv('../../data/clean/trisma2016_cleaned_text_by_lga_and_time_of_day.csv')